In [10]:
#Установка необходимых библиотек
!pip install selenium
!pip install webdriver-manager
!pip install pandas
!pip install matplotlib
!pip install seaborn
!pip install streamlit
!pip install wordcloud

In [24]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/136.0.7103.93 Safari/537.36")
options.headless = True  

service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options) #Подключаем драйвер
movies = [] #Списки, которые в будущем станут основой датафрейма
all_years = []
genres = []
rates = []
countries = []

try:
    for i in range(1, 14):
        url = f'https://www.kinopoisk.ru/lists/movies/oscar-best-film-nominees/?page={i}' # Открываем страницу
        driver.get(url)
        time.sleep(20) # Время для прохождения капчи
        # Прокручиваем страницу, чтобы прогрузить все карточки
        for i in range(4):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(1)
        
        # Получаем данные
        movie = driver.find_elements(By.CSS_SELECTOR, '.styles_mainTitle__IFQyZ')
        years = driver.find_elements(By.CSS_SELECTOR, '.desktop-list-main-info_secondaryText__M_aus')
        extra_inf = driver.find_elements(By.CSS_SELECTOR, '.desktop-list-main-info_truncatedText__IMQRP')
        rate = driver.find_elements(By.CSS_SELECTOR, '.styles_kinopoiskValue__nkZEC')
        
        for i in rate: # Заносим результаты в списки в читаемом виде
            rates.append(i.text)
        for mov in movie:
            movies.append(mov.text)
        for year in years:
            all_years.append(str(year.text)[2:6])
        for i in extra_inf:
            if '•' in i.text:
                divide = str(i.text).split(' • ')
                countries.append(divide[0])
                genres.append((divide[1].split())[0])
        time.sleep(5)
finally:
    pass

win_movies = [] #Информация о наличии Оскара за лучший фильм
try:
    win_url = 'https://www.kinopoisk.ru/lists/movies/oscar-best-film/'
    driver.get(win_url)
        
    wait = WebDriverWait(driver, 10) #Ожидаем загрузки элементов
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.styles_mainTitle__IFQyZ')))
    movie = driver.find_elements(By.CSS_SELECTOR, '.styles_mainTitle__IFQyZ')
    for mov in movie:
        win_movies.append(mov.text)
    time.sleep(5)

    win_url = 'https://www.kinopoisk.ru/lists/movies/oscar-best-film/?page=2'
    driver.get(win_url)
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.styles_mainTitle__IFQyZ')))
    movie = driver.find_elements(By.CSS_SELECTOR, '.styles_mainTitle__IFQyZ')
    for mov in movie:
        win_movies.append(mov.text)
    time.sleep(5)
finally:
    driver.quit()

In [19]:
#Этап сбора данных завершен. Теперь переведем получившиеся списки в датафрейм

In [26]:
import pandas as pd

data = []

for i in range(611):
    if movies[i] in win_movies:
        win_status = True
    else: 
        win_status = False
    data.append([movies[i], win_status, all_years[i], genres[i], countries[i], rates[i]])
df = pd.DataFrame(data=data,  # каждый список - один ряд
                 index=[i for i in range(1, 612)],  # сколько рядов, столько и индексов
                 columns=["Название", "Победитель", "Год выпуска", "Жанр", "Страна выпуска", "Рейтинг"]  # сколько элементов в ряде, столько и колонок
                 )

df
df.to_csv('parsed_data.csv', index=False)